# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from functools import partial



In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)

    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df

    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    while(True):
        yield df.sample(100).reset_index(drop=True)

gen = partial_df()

In [3]:
# Initialize generator for batches
gen = partial_df()

In [4]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2013,Hyundai,Elantra,GLS,Sedan,automatic,5npdh4ae5dh250547,fl,3.4,51003.0,gray,gray,avis corporation,10450,10600,Tue Feb 17 2015 01:30:00 GMT-0800 (PST)
1,2013,Chevrolet,Express,LS 3500,Van,automatic,1gazg1fg7d1158552,fl,4.4,36251.0,white,gray,enterprise veh exchange/rental,20200,21000,Tue Feb 10 2015 01:30:00 GMT-0800 (PST)
2,2012,Kia,Optima,LX,Sedan,automatic,5xxgm4a79cg062571,wa,4.4,60927.0,red,beige,repo remarketing/verity credit union,10250,12500,Wed Jan 07 2015 12:30:00 GMT-0800 (PST)
3,2008,Suzuki,SX4,Sport,Sedan,automatic,js2yc414285103024,fl,1.9,93758.0,red,—,credit acceptance corp/vrs/southfield,3025,2350,Thu Feb 19 2015 17:00:00 GMT-0800 (PST)
4,2013,Toyota,Avalon,XLE,Sedan,NaN,4t1bk1eb6du017253,nc,3.9,45488.0,silver,black,united fcu,18200,18600,Tue Jan 20 2015 01:15:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2013,Audi,A4,2.0T Premium quattro,Sedan,automatic,waubfafl7dn030472,tx,4.3,30209.0,silver,black,silver car,23500,23700,Thu Feb 19 2015 03:00:00 GMT-0800 (PST)
96,2010,Ford,Flex,SEL,Wagon,automatic,2fmgk5cc0aba31790,ga,3.6,100073.0,black,black,rick hendrick chevrolet,10200,10600,Tue Jan 06 2015 13:00:00 GMT-0800 (PST)
97,2006,Toyota,Sienna,XLE Limited 7-Passenger,minivan,automatic,5tdza22c26s580073,nv,1.9,105309.0,blue,gray,findlay kia,7750,7000,Fri May 29 2015 05:00:00 GMT-0700 (PDT)
98,2011,Ford,Flex,Limited,Wagon,automatic,2fmgk5dc5bbd14699,ca,2.9,72017.0,burgundy,beige,tradewinds remarketing,17850,13900,Wed Jan 28 2015 04:30:00 GMT-0800 (PST)


In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from functools import partial
regressor = SGDRegressor(max_iter=1000, tol=1e-3)


batch_generation = partial_df()
for _ in range(50):
    X_batches = next(batch_generation)[['year', 'odometer']]
    y_batches = next(batch_generation)['sellingprice']

    # Impute missing values
    imputer = SimpleImputer(strategy='median')
    X_batch_imputed = imputer.fit_transform(X_batches)

    regressor.partial_fit(X_batch_imputed, y_batches)

X_test, y_test = test_df()


imputer = SimpleImputer(strategy='median')
X_test_imputed = imputer.fit_transform(X_test[['year', 'odometer']])

test_predictions = regressor.predict(X_test_imputed)

# Calculate performance metrics (e.g., RMSE)
rmse = np.sqrt(np.mean((test_predictions - y_test) ** 2))
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 6.901784453712306e+18


In [6]:
test_predictions

array([7.83265290e+18, 2.83797062e+18, 1.69299716e+18, ...,
       1.81707839e+18, 4.65557066e+17, 1.45292207e+18])